# Wikipedia LLM Training Notebook

This notebook contains the complete pipeline for training a Wikipedia-based language model using Phi-2 with LoRA fine-tuning.

## Table of Contents
1. [Package Installation](#Package-Installation)
2. [GPU Check](#GPU-Check)
3. [Data Download](#Data-Download)
4. [Data Preparation](#Data-Preparation)
5. [Model Setup](#Model-Setup)
6. [LoRA Configuration](#LoRA-Configuration)
7. [Training Progress Check](#Training-Progress-Check)
8. [Model Training](#Model-Training)
9. [VRAM Cleanup](#VRAM-Cleanup)
10. [Model Testing](#Model-Testing)
11. [Chat Interface](#Chat-Interface)
12. [Training Restart](#Training-Restart)
13. [Hyperparameter Tuning](#Hyperparameter-Tuning)
14. [Model Evaluation](#Model-Evaluation)
15. [Model Deployment](#Model-Deployment)
16. [Data Analysis](#Data-Analysis)

## Package Installation

Installs all required Python packages for the training pipeline, including PyTorch, Transformers, PEFT, and other dependencies.

In [ ]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/nightly/cu128
!pip install transformers datasets peft accelerate bitsandbytes trl tqdm protobuf scipy sentencepiece psutil matplotlib optuna mlflow rouge-score nltk wordcloud seaborn pandas

## GPU Check

Verifies CUDA availability and GPU information to ensure the system is ready for GPU-accelerated training.

In [ ]:
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'N/A'}")

## Data Download

Downloads a subset of the Wikipedia dataset (100,000 articles) from Hugging Face and saves it locally for training.

In [ ]:
# download_wikipedia_working.py
from datasets import load_dataset
import os

os.makedirs("data", exist_ok=True)

print("Downloading Wikipedia dataset (this will take a while)...")
dataset = load_dataset(
    "wikimedia/wikipedia",
    "20231101.en",  # November 2023 snapshot
    split="train",
    streaming=False  # Set to True if to stream instead of download all
)

# Take subset for testing (adjust as needed)
print("Creating subset...")
subset = dataset.select(range(min(100000, len(dataset))))

print("Saving dataset locally...")
subset.save_to_disk("data/wikipedia_100k")
print(f"Dataset saved! Total articles: {len(subset)}")
print(f"Sample article title: {subset[0]['title']}")
print(f"Sample text: {subset[0]['text'][:200]}")

## Data Preparation

Processes the raw Wikipedia data with text cleaning, quality filtering, and multiple instruction formats for better training data quality and diversity.

In [ ]:
# prepare_data.py
from datasets import load_from_disk
import random
import re

# Load saved dataset
dataset = load_from_disk("data/wikipedia_100k")

def clean_text(text):
    """Clean and preprocess text"""
    # Remove HTML tags
    text = re.sub(r'<[^>]+>', '', text)
    # Remove excessive whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    # Remove URLs
    text = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', text)
    # Remove wiki markup
    text = re.sub(r'\[\[([^\]|]*\|)?([^\]]*)\]\]', r'\2', text)  # Remove [[links]]
    text = re.sub(r"''+", '', text)  # Remove italic/bold markup
    return text

def format_for_training(example):
    """Convert Wikipedia articles to multiple instruction formats"""
    text = clean_text(example['text'])
    title = example['title']
    
    # Skip very short articles
    if len(text) < 200:
        return {"text": ""}  # Will be filtered out
    
    # Create multiple instruction formats for variety
    formats = [
        f"### Instruction:\nProvide information about {title}.\n\n### Response:\n{text[:1000]}",
        f"### Instruction:\nExplain what {title} is.\n\n### Response:\n{text[:1000]}",
        f"### Instruction:\nTell me about {title}.\n\n### Response:\n{text[:1000]}",
        f"### Instruction:\nGive me details on {title}.\n\n### Response:\n{text[:1000]}"
    ]
    
    # Randomly select one format
    return {"text": random.choice(formats)}

# Format dataset
print("Formatting and cleaning dataset...")
formatted_dataset = dataset.map(
    format_for_training,
    remove_columns=dataset.column_names
)

# Filter out empty entries
formatted_dataset = formatted_dataset.filter(lambda x: len(x['text']) > 50)

# Split into train/validation
split_dataset = formatted_dataset.train_test_split(test_size=0.05, seed=42)

# Save formatted data
split_dataset.save_to_disk("data/formatted_wikipedia")
print("Data preparation complete!")
print(f"Training samples: {len(split_dataset['train'])}")
print(f"Validation samples: {len(split_dataset['test'])}")
print(f"Filtered out {len(dataset) - len(formatted_dataset)} short/invalid articles")

## Model Setup

Loads the Phi-2 model with 4-bit quantization, performs system resource checks, memory estimation, and provides model merging capabilities for optimized inference.

In [ ]:
# model_setup.py
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch
import psutil

# Choose a base model (recommendations for your hardware)
MODEL_OPTIONS = {
    "small": "TinyLlama/TinyLlama-1.1B-Chat-v1.0",  # 1.1B - easiest to train
    "medium": "microsoft/phi-2",  # 2.7B - good balance
    "large": "mistralai/Mistral-7B-v0.1"  # 7B - needs quantization
}

model_name = MODEL_OPTIONS["medium"]  # Start with Phi-2

# Quantization config for memory efficiency
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

def estimate_memory_usage(model_name, quantized=True):
    """Estimate memory usage for the model"""
    # Rough estimates (in GB)
    base_sizes = {
        "TinyLlama/TinyLlama-1.1B-Chat-v1.0": 2.2,
        "microsoft/phi-2": 5.0,
        "mistralai/Mistral-7B-v0.1": 14.0
    }
    
    base_size = base_sizes.get(model_name, 5.0)
    if quantized:
        estimated_vram = base_size * 0.3  # 4-bit quantization reduces to ~30%
    else:
        estimated_vram = base_size
    
    return estimated_vram

def check_system_resources():
    """Check available system resources"""
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1024**3 if torch.cuda.is_available() else 0
    cpu_memory = psutil.virtual_memory().total / 1024**3
    
    print(f"Available GPU Memory: {gpu_memory:.1f} GB")
    print(f"Available CPU Memory: {cpu_memory:.1f} GB")
    return gpu_memory, cpu_memory

# Check resources before loading
print("Checking system resources...")
gpu_mem, cpu_mem = check_system_resources()

estimated_vram = estimate_memory_usage(model_name, quantized=True)
print(f"Estimated VRAM usage: {estimated_vram:.1f} GB")

if gpu_mem > 0 and estimated_vram > gpu_mem * 0.8:
    print("WARNING: Estimated VRAM usage is high. Consider using smaller model or more aggressive quantization.")
elif gpu_mem == 0:
    print("WARNING: No GPU detected. Training will be very slow on CPU.")

# Load model
print(f"\nLoading model: {model_name}")
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

print("Model loaded successfully!")
print(f"Model size: ~{sum(p.numel() for p in model.parameters()) / 1e9:.2f}B parameters")

## Optional: Model Merging

This section provides functionality to merge LoRA weights back into the base model for faster inference after training.

### Usage Instructions:

1. Train your LoRA model using the training cells below.
2. After training completes, run the code cell below to merge the LoRA weights.
3. Provide the base model (loaded above), the path to your trained LoRA weights, and a save path for the merged model.
4. Example: `merged_model = merge_lora_weights(model, './wikipedia_model/final', './wikipedia_model/merged')`
5. The merged model will be saved and can be used for faster inference without LoRA adapters.

**Note:** This step is optional. If you prefer to keep the model in LoRA format for flexibility, you can skip this.

In [ ]:
# Model merging capability
def merge_lora_weights(base_model, lora_path, save_path):
    """Merge LoRA weights back into base model for faster inference"""
    from peft import PeftModel
    
    print("Loading LoRA weights...")
    lora_model = PeftModel.from_pretrained(base_model, lora_path)
    
    print("Merging weights...")
    merged_model = lora_model.merge_and_unload()
    
    print(f"Saving merged model to {save_path}...")
    merged_model.save_pretrained(save_path)
    tokenizer.save_pretrained(save_path)
    
    print("Model merging complete!")
    return merged_model

# Example usage (uncomment to use):
# merged_model = merge_lora_weights(model, "./wikipedia_model/final", "./wikipedia_model/merged")

## LoRA Configuration

Configures Low-Rank Adaptation with dynamic rank scaling, expanded target modules, and enhanced regularization for optimal fine-tuning performance.

In [ ]:
# lora_config.py
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
import torch

# Dynamic LoRA rank based on model size
def get_dynamic_lora_rank(model):
    """Determine LoRA rank based on model parameter count"""
    param_count = sum(p.numel() for p in model.parameters())
    
    if param_count < 2e9:  # < 2B params
        rank = 8
    elif param_count < 7e9:  # 2-7B params
        rank = 16
    else:  # > 7B params
        rank = 32
    
    return rank

# Prepare model for training
model = prepare_model_for_kbit_training(model)

# Get dynamic rank
lora_rank = get_dynamic_lora_rank(model)
print(f"Using LoRA rank: {lora_rank} (based on model size)")

# Enhanced LoRA configuration with more target modules
lora_config = LoraConfig(
    r=lora_rank,  # Dynamic rank
    lora_alpha=lora_rank * 2,  # Typically 2x rank
    target_modules=[
        "q_proj",   # Query projection
        "k_proj",   # Key projection  
        "v_proj",   # Value projection
        "o_proj",   # Output projection
        "gate_proj",  # Gate projection (for some models)
        "up_proj",    # Up projection (for some models)
        "down_proj",  # Down projection (for some models)
    ],
    lora_dropout=0.1,  # Slightly higher dropout for better regularization
    bias="none",
    task_type="CAUSAL_LM"
)

# Apply LoRA
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

# LoRA dropout scheduling (optional - requires custom training loop)
# This would need to be implemented in the training loop to decrease dropout over time
def get_scheduled_dropout(epoch, max_epochs, initial_dropout=0.1, final_dropout=0.01):
    """Linearly decrease dropout from initial to final over training"""
    return initial_dropout - (initial_dropout - final_dropout) * (epoch / max_epochs)

print(f"\nLoRA Configuration Summary:")
print(f"- Rank: {lora_rank}")
print(f"- Alpha: {lora_rank * 2}")
print(f"- Target Modules: {len(lora_config.target_modules)} modules")
print(f"- Dropout: {lora_config.lora_dropout}")
print("- Bias: none")
print("- Task: Causal LM")

## Training Progress Check

Analyzes existing checkpoints with loss curve visualization, training speed metrics, and live model performance preview through sample generation.

In [ ]:
# check_progress.py
import os
import glob
import json
import matplotlib.pyplot as plt
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
import torch

def check_training_progress():
    """Check current training progress with enhanced metrics and visualization"""
    
    output_dir = "./wikipedia_model"
    
    # Find all checkpoints
    checkpoints = glob.glob(os.path.join(output_dir, "checkpoint-*"))
    
    if not checkpoints:
        print("No checkpoints found. Training hasn't started yet.")
        return
    
    # Sort by step number
    checkpoints = sorted(checkpoints, key=lambda x: int(x.split("-")[-1]))
    
    print(f"Total checkpoints found: {len(checkpoints)}")
    print("\nCheckpoint history:")
    
    # Collect data for plotting
    steps = []
    losses = []
    learning_rates = []
    eval_losses = []
    
    for cp in checkpoints:
        step = cp.split("-")[-1]
        
        # Try to read trainer_state.json for more info
        state_file = os.path.join(cp, "trainer_state.json")
        if os.path.exists(state_file):
            with open(state_file, 'r') as f:
                state = json.load(f)
                epoch = state.get('epoch', 'N/A')
                print(f"  - Step {step} (Epoch {epoch:.2f})")
                
                # Collect training metrics
                log_history = state.get('log_history', [])
                training_entries = [entry for entry in log_history if 'loss' in entry]
                eval_entries = [entry for entry in log_history if 'eval_loss' in entry]
                
                if training_entries:
                    last_train = training_entries[-1]
                    steps.append(int(step))
                    losses.append(last_train.get('loss', 0))
                    learning_rates.append(last_train.get('learning_rate', 0))
                
                if eval_entries:
                    last_eval = eval_entries[-1]
                    eval_losses.append(last_eval.get('eval_loss', 0))
        else:
            print(f"  - Step {step}")
    
    # Latest checkpoint
    latest = checkpoints[-1]
    print(f"\nLatest checkpoint: {latest}")
    
    # Read detailed info from latest
    state_file = os.path.join(latest, "trainer_state.json")
    if os.path.exists(state_file):
        with open(state_file, 'r') as f:
            state = json.load(f)
            print(f"\nDetailed Progress:")
            print(f"  Current Step: {state.get('global_step', 'N/A')}")
            print(f"  Current Epoch: {state.get('epoch', 'N/A'):.2f}")
            
            # Get latest training metrics (find last entry with training loss)
            log_history = state.get('log_history', [])
            training_entries = [entry for entry in log_history if 'loss' in entry]
            eval_entries = [entry for entry in log_history if 'eval_loss' in entry]
            
            if training_entries:
                last_training = training_entries[-1]
                training_loss = last_training.get('loss', 'N/A')
                learning_rate = last_training.get('learning_rate', 'N/A')
            else:
                training_loss = 'N/A'
                learning_rate = 'N/A'
            
            if eval_entries:
                last_eval = eval_entries[-1]
                validation_loss = last_eval.get('eval_loss', 'N/A')
            else:
                validation_loss = 'N/A'
            
            print(f"  Training Loss: {training_loss}")
            print(f"  Validation Loss: {validation_loss}")
            print(f"  Learning Rate: {learning_rate}")
            
            # Calculate progress percentage
            max_steps = state.get('max_steps', None)
            if max_steps:
                current_step = state.get('global_step', 0)
                percentage = (current_step / max_steps) * 100
                print(f"  Progress: {percentage:.1f}% ({current_step}/{max_steps} steps)")
                
                # Estimate training speed
                if len(steps) > 1:
                    steps_per_hour = (steps[-1] - steps[0]) / ((len(steps) - 1) * 0.1)  # Rough estimate
                    print(f"  Estimated Speed: ~{steps_per_hour:.1f} steps/hour")
            else:
                print("  Progress: Unable to calculate (max_steps not available)")
            
            # Plot loss curves if we have data
            if len(losses) > 1:
                plt.figure(figsize=(12, 4))
                
                plt.subplot(1, 3, 1)
                plt.plot(steps, losses, 'b-', label='Training Loss')
                plt.xlabel('Steps')
                plt.ylabel('Loss')
                plt.title('Training Loss Curve')
                plt.legend()
                
                plt.subplot(1, 3, 2)
                plt.plot(steps, learning_rates, 'r-', label='Learning Rate')
                plt.xlabel('Steps')
                plt.ylabel('Learning Rate')
                plt.title('Learning Rate Schedule')
                plt.legend()
                
                plt.subplot(1, 3, 3)
                if eval_losses:
                    eval_steps = steps[:len(eval_losses)]  # Assume eval at same steps
                    plt.plot(eval_steps, eval_losses, 'g-', label='Validation Loss')
                plt.xlabel('Steps')
                plt.ylabel('Loss')
                plt.title('Validation Loss')
                plt.legend()
                
                plt.tight_layout()
                plt.show()
            
            # Show recent training history (last 3 training entries with validation loss)
            if training_entries:
                print(f"\nRecent Training History (last {min(3, len(training_entries))} entries):")
                for entry in training_entries[-3:]:
                    step = entry.get('step', 'N/A')
                    loss = entry.get('loss', 'N/A')
                    lr = entry.get('learning_rate', 'N/A')
                    epoch = entry.get('epoch', 'N/A')
                    
                    # Find corresponding validation loss for this step
                    val_loss = 'N/A'
                    for eval_entry in eval_entries:
                        if eval_entry.get('step') == step:
                            val_loss = eval_entry.get('eval_loss', 'N/A')
                            break
                    
                    print(f"    Step {step}: Train Loss={loss}, Val Loss={val_loss}, LR={lr}, Epoch={epoch:.2f}")
            else:
                print("\nNo training history available yet.")
            
            # Generate sample output from latest checkpoint
            print(f"\nSample Generation from Latest Checkpoint:")
            try:
                # Load model for inference
                base_model = AutoModelForCausalLM.from_pretrained(
                    "microsoft/phi-2",
                    torch_dtype=torch.float16,
                    device_map="auto"
                )
                lora_model = PeftModel.from_pretrained(base_model, latest)
                test_tokenizer = AutoTokenizer.from_pretrained(latest)
                
                # Test prompt
                prompt = "### Instruction:\nProvide information about Artificial Intelligence.\n\n### Response:\n"
                inputs = test_tokenizer(prompt, return_tensors="pt").to(lora_model.device)
                
                with torch.no_grad():
                    outputs = lora_model.generate(
                        **inputs,
                        max_length=100,
                        temperature=0.7,
                        do_sample=True,
                        pad_token_id=test_tokenizer.eos_token_id
                    )
                
                response = test_tokenizer.decode(outputs[0], skip_special_tokens=True)
                # Extract just the response part
                answer = response.split("### Response:\n")[-1][:200] + "..."
                print(f"AI: {answer}")
                
                # Clean up
                del base_model, lora_model
                torch.cuda.empty_cache()
                
            except Exception as e:
                print(f"Could not generate sample: {e}")
    
    print("\nTo resume training, simply run: python train.py")

if __name__ == "__main__":
    check_training_progress()

## Model Training

Executes the main training loop with optimized settings including gradient checkpointing, mixed precision, and automatic checkpoint resumption.

In [ ]:
# train.py
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling
from datasets import load_from_disk
import os
import glob
import gc
import time

def clear_vram():
    """Clear VRAM before training"""
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        gc.collect()
        print("VRAM cleared")

# Call it before loading model
clear_vram()

# Load formatted data
dataset = load_from_disk("data/formatted_wikipedia")

# Tokenize function
def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        truncation=True,
        max_length=512,  # Adjust based on your needs
        padding="max_length"
    )

tokenized_dataset = dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=dataset["train"].column_names
)

# Data collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # Causal LM, not masked
)

# Training arguments
training_args = TrainingArguments(
    output_dir="./wikipedia_model",
    num_train_epochs=3,
    per_device_train_batch_size=8,     # Adjust based on GPU memory
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=16,     # Effective batch size = 128
    learning_rate=2e-4,
    fp16=True,                         # Mixed precision training
    save_steps=100,                    # Save every 100 steps
    eval_steps=100,                    # Evaluate every 100 steps
    logging_steps=100,
    eval_strategy="steps",
    save_total_limit=3,                # Keep only last 3 checkpoints (saves disk space)
    load_best_model_at_end=True,
    warmup_steps=100,
    lr_scheduler_type="cosine",
    optim="paged_adamw_8bit",          # Memory efficient optimizer
    resume_from_checkpoint=True,       # Auto-resume from latest checkpoint
    gradient_checkpointing=True,       # Enable gradient checkpointing for memory efficiency
    dataloader_num_workers=2,          # Use 2 workers for data loading
    logging_dir="./logs",              # Directory for TensorBoard logs
    report_to=["tensorboard"],         # Enable TensorBoard logging
    seed=42,                           # Set seed for reproducibility
    weight_decay=0.01,                 # Add weight decay for regularization
    max_grad_norm=1.0,                 # Gradient clipping
)

# Initialize trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    data_collator=data_collator,
)

# Check for existing checkpoints
def find_latest_checkpoint(output_dir):
    """Find the latest checkpoint in the output directory"""
    checkpoints = glob.glob(os.path.join(output_dir, "checkpoint-*"))
    if not checkpoints:
        return None
    # Sort by step number
    checkpoints = sorted(checkpoints, key=lambda x: int(x.split("-")[-1]))
    latest = checkpoints[-1]
    return latest

# Look for existing checkpoint
checkpoint_path = find_latest_checkpoint("./wikipedia_model")

start_time = time.time()

if checkpoint_path:
    print(f"Found existing checkpoint: {checkpoint_path}")
    print("Resuming training from checkpoint...")
    print(f"Progress: Step {checkpoint_path.split('-')[-1]}")
    
    # Resume training from checkpoint
    try:
        trainer.train(resume_from_checkpoint=checkpoint_path)
    except KeyboardInterrupt:
        print("Training interrupted by user")
else:
    print("No existing checkpoint found. Starting fresh training...")
    # Start training from scratch
    try:
        trainer.train()
    except KeyboardInterrupt:
        print("Training interrupted by user")

end_time = time.time()
training_duration = end_time - start_time

# Set model to eval mode
# model.eval() ## training sets it to eval automatically

# Save final model
print("Saving final model...")
trainer.save_model("./wikipedia_model/final")
tokenizer.save_pretrained("./wikipedia_model/final")
# Also save the LoRA weights explicitly
model.save_pretrained("./wikipedia_model/final")

# Training summary
print("\n" + "="*50)
print("TRAINING SUMMARY")
print("="*50)
print(f"Total training time: {training_duration:.2f} seconds ({training_duration/3600:.2f} hours)")
print(f"Model saved to: ./wikipedia_model/final")
print(f"TensorBoard logs: ./logs (run 'tensorboard --logdir ./logs' to view)")
print(f"Checkpoints saved every {training_args.save_steps} steps")
print(f"Final step: {trainer.state.global_step}")
print(f"Final epoch: {trainer.state.epoch:.2f}")
print("Training complete!")
print("="*50)

## VRAM Cleanup

Clears GPU memory cache and forces garbage collection to free up VRAM after training or testing.

In [ ]:
# restart_training.py
import os
import shutil

def restart_training():
    """Delete all training files to restart from scratch"""
    
    # Directories to delete
    dirs_to_delete = ["./wikipedia_model", "./logs"]
    
    for dir_path in dirs_to_delete:
        if os.path.exists(dir_path):
            print(f"Deleting {dir_path}...")
            shutil.rmtree(dir_path)
            print(f"Deleted {dir_path}")
        else:
            print(f"{dir_path} does not exist")
    
    print("\nAll training files deleted. You can now run the training cell to start fresh.")

if __name__ == "__main__":
    # Ask for confirmation
    confirm = input("This will delete all training checkpoints and logs. Are you sure? (yes/no): ")
    if confirm.lower() == 'yes':
        restart_training()
    else:
        print("Restart cancelled.")

In [ ]:
# clear_vram.py
import torch
import gc

def clear_vram():
    """Clear VRAM completely"""
    print("Clearing VRAM...")
    
    # Clear PyTorch cache
    torch.cuda.empty_cache()
    
    # Force garbage collection
    gc.collect()
    
    # Print VRAM usage
    if torch.cuda.is_available():
        print(f"VRAM allocated: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")
        print(f"VRAM reserved: {torch.cuda.memory_reserved() / 1024**3:.2f} GB")
    
    print("VRAM cleared!")

if __name__ == "__main__":
    clear_vram()

## Model Testing

Loads a trained checkpoint and tests the model's ability to generate informative responses about various topics.

In [ ]:
# test_model.py
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
import torch
import numpy as np
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
import glob
import os
import json

def calculate_perplexity(model, tokenizer, text, max_length=512):
    """Calculate perplexity for a given text"""
    inputs = tokenizer(text, return_tensors="pt", max_length=max_length, truncation=True).to(model.device)
    
    with torch.no_grad():
        outputs = model(**inputs, labels=inputs["input_ids"])
        loss = outputs.loss
        perplexity = torch.exp(loss).item()
    
    return perplexity

def calculate_bleu_rouge(generated, reference):
    """Calculate BLEU and ROUGE scores"""
    # BLEU
    smoothing = SmoothingFunction().method4
    bleu = sentence_bleu([reference.split()], generated.split(), smoothing_function=smoothing)
    
    # ROUGE
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    rouge_scores = scorer.score(reference, generated)
    
    return bleu, rouge_scores

def load_checkpoint_model(checkpoint_path):
    """Load model from checkpoint"""
    base_model = AutoModelForCausalLM.from_pretrained(
        "microsoft/phi-2",
        torch_dtype=torch.float16,
        device_map="auto"
    )
    model = PeftModel.from_pretrained(base_model, checkpoint_path)
    tokenizer = AutoTokenizer.from_pretrained(checkpoint_path)
    return model, tokenizer

def evaluate_model(model, tokenizer, test_prompts):
    """Comprehensive model evaluation"""
    results = {
        'perplexity': [],
        'bleu': [],
        'rouge1': [],
        'rouge2': [],
        'rougeL': [],
        'responses': []
    }
    
    for prompt_data in test_prompts:
        prompt = prompt_data['prompt']
        reference = prompt_data['reference']
        
        # Generate response
        inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_length=len(inputs['input_ids'][0]) + 100,
                temperature=0.7,
                do_sample=True,
                pad_token_id=tokenizer.eos_token_id
            )
        
        response = tokenizer.decode(outputs[0], skip_special_tokens=True)
        generated_answer = response.split("### Response:\n")[-1].strip()
        
        # Calculate metrics
        perplexity = calculate_perplexity(model, tokenizer, generated_answer)
        bleu, rouge_scores = calculate_bleu_rouge(generated_answer, reference)
        
        results['perplexity'].append(perplexity)
        results['bleu'].append(bleu)
        results['rouge1'].append(rouge_scores['rouge1'].fmeasure)
        results['rouge2'].append(rouge_scores['rouge2'].fmeasure)
        results['rougeL'].append(rouge_scores['rougeL'].fmeasure)
        results['responses'].append({
            'prompt': prompt,
            'generated': generated_answer,
            'reference': reference
        })
    
    # Calculate averages
    for key in ['perplexity', 'bleu', 'rouge1', 'rouge2', 'rougeL']:
        results[f'avg_{key}'] = np.mean(results[key])
    
    return results

# Test prompts with references for evaluation
test_prompts = [
    {
        'prompt': "### Instruction:\nProvide information about Python programming language.\n\n### Response:\n",
        'reference': "Python is a high-level programming language known for its simplicity and readability. It supports multiple programming paradigms including procedural, object-oriented, and functional programming."
    },
    {
        'prompt': "### Instruction:\nExplain what Artificial Intelligence is.\n\n### Response:\n",
        'reference': "Artificial Intelligence (AI) is the simulation of human intelligence processes by machines, especially computer systems. It includes learning, reasoning, and self-correction."
    },
    {
        'prompt': "### Instruction:\nTell me about Machine Learning.\n\n### Response:\n",
        'reference': "Machine Learning is a subset of AI that enables computers to learn and improve from experience without being explicitly programmed. It uses algorithms to identify patterns in data."
    },
    {
        'prompt': "### Instruction:\nWhat is the capital of France?\n\n### Response:\n",
        'reference': "The capital of France is Paris, which is located in the north-central part of the country along the Seine River."
    },
    {
        'prompt': "### Instruction:\nDescribe the process of photosynthesis.\n\n### Response:\n",
        'reference': "Photosynthesis is the process by which plants convert light energy into chemical energy. It involves chlorophyll absorbing sunlight and using it to convert carbon dioxide and water into glucose and oxygen."
    }
]

# Find all checkpoints for comparison
checkpoints = glob.glob("./wikipedia_model/checkpoint-*")
checkpoints = sorted(checkpoints, key=lambda x: int(x.split("-")[-1]))

if checkpoints:
    print("Available checkpoints:")
    for i, cp in enumerate(checkpoints):
        print(f"{i+1}. {cp}")
    
    # Test latest checkpoint
    latest_checkpoint = checkpoints[-1]
    print(f"\nTesting latest checkpoint: {latest_checkpoint}")
    
    model, tokenizer = load_checkpoint_model(latest_checkpoint)
    
    # Run evaluation
    results = evaluate_model(model, tokenizer, test_prompts)
    
    print("\n" + "="*60)
    print("MODEL EVALUATION RESULTS")
    print("="*60)
    print(f"Average Perplexity: {results['avg_perplexity']:.2f}")
    print(f"Average BLEU Score: {results['avg_bleu']:.4f}")
    print(f"Average ROUGE-1 F1: {results['avg_rouge1']:.4f}")
    print(f"Average ROUGE-2 F1: {results['avg_rouge2']:.4f}")
    print(f"Average ROUGE-L F1: {results['avg_rougeL']:.4f}")
    print("="*60)
    
    # Show sample responses
    print("\nSample Responses:")
    for i, resp in enumerate(results['responses'][:3]):
        print(f"\n{i+1}. Prompt: {resp['prompt'].split('### Instruction:\n')[1].split('\n\n### Response:\n')[0]}")
        print(f"   Generated: {resp['generated'][:100]}...")
        print(f"   Reference: {resp['reference'][:100]}...")
    
    # Checkpoint comparison (if multiple checkpoints exist)
    if len(checkpoints) > 1:
        print(f"\nCheckpoint Comparison (last {min(3, len(checkpoints))} checkpoints):")
        comparison_results = []
        
        for cp in checkpoints[-3:]:
            try:
                model_comp, tokenizer_comp = load_checkpoint_model(cp)
                results_comp = evaluate_model(model_comp, tokenizer_comp, test_prompts[:2])  # Quick comparison
                step = cp.split("-")[-1]
                comparison_results.append({
                    'step': step,
                    'perplexity': results_comp['avg_perplexity'],
                    'bleu': results_comp['avg_bleu']
                })
                del model_comp
                torch.cuda.empty_cache()
            except:
                continue
        
        for res in comparison_results:
            print(f"Step {res['step']}: Perplexity={res['perplexity']:.2f}, BLEU={res['bleu']:.4f}")
    
    # Cleanup
    del model
    torch.cuda.empty_cache()
    
else:
    print("No checkpoints found. Please train the model first.")

print("\nModel testing complete!")

## Chat Interface

Creates an interactive chat interface for real-time conversation with the trained Wikipedia model.

In [ ]:
# interface.py
from transformers import pipeline
import torch
import re

class SafeChatInterface:
    def __init__(self, model_path="./wikipedia_model/checkpoint-100"):
        # Create text generation pipeline
        self.pipe = pipeline(
            "text-generation",
            model=model_path,
            tokenizer=model_path,
            device=0 if torch.cuda.is_available() else -1,
            torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32
        )
        
        # Conversation history
        self.history = []
        self.max_history = 10
        
        # Safety patterns
        self.safety_patterns = [
            r'\b(hate|violence|kill|murder|suicide)\b',
            r'\b(drug|weapon|nuclear)\b.*\b(make|build|create)\b',
            r'\b(hack|exploit|crack)\b.*\b(password|system|account)\b'
        ]
    
    def check_safety(self, text):
        """Check if response contains unsafe content"""
        text_lower = text.lower()
        for pattern in self.safety_patterns:
            if re.search(pattern, text_lower):
                return False, "Response contains potentially unsafe content."
        return True, ""
    
    def filter_response(self, response):
        """Filter and clean the response"""
        # Extract just the response part
        if "### Response:\n" in response:
            answer = response.split("### Response:\n")[-1]
        else:
            answer = response
        
        # Remove any instruction-like prefixes that might leak through
        answer = re.sub(r'### Instruction:.*?\n', '', answer)
        
        # Limit length
        if len(answer) > 1000:
            answer = answer[:1000] + "..."
        
        return answer.strip()
    
    def generate_streaming(self, prompt, max_length=300):
        """Generate response with simulated streaming"""
        inputs = self.pipe.tokenizer(prompt, return_tensors="pt")
        if torch.cuda.is_available():
            inputs = {k: v.to('cuda') for k, v in inputs.items()}
        
        generated_tokens = inputs['input_ids'][0].tolist()
        
        print("Assistant: ", end="", flush=True)
        
        for i in range(max_length - len(generated_tokens)):
            with torch.no_grad():
                outputs = self.pipe.model(**inputs)
                next_token_logits = outputs.logits[:, -1, :]
                
                # Apply temperature and top-p sampling
                next_token_logits = next_token_logits / 0.7
                probs = torch.softmax(next_token_logits, dim=-1)
                
                # Get top tokens
                top_probs, top_tokens = torch.topk(probs, 50, dim=-1)
                top_probs = top_probs / top_probs.sum()
                
                # Sample from top tokens
                next_token = torch.multinomial(top_probs, 1).item()
                next_token = top_tokens[0, next_token].item()
                
                if next_token == self.pipe.tokenizer.eos_token_id:
                    break
                
                generated_tokens.append(next_token)
                token_text = self.pipe.tokenizer.decode([next_token], skip_special_tokens=True)
                print(token_text, end="", flush=True)
                
                # Update inputs for next iteration
                inputs['input_ids'] = torch.tensor([generated_tokens]).to(inputs['input_ids'].device)
        
        print()  # New line
        return self.pipe.tokenizer.decode(generated_tokens, skip_special_tokens=True)
    
    def chat(self):
        print("Wikipedia LLM - Enhanced Chat Interface")
        print("=" * 50)
        print("Features: Conversation history, safety checks, streaming responses")
        print("Commands: 'history' to view chat history, 'clear' to reset, 'quit' to exit")
        print("=" * 50)
        
        while True:
            query = input("\nYou: ").strip()
            
            if query.lower() == 'quit':
                break
            elif query.lower() == 'history':
                print("\nConversation History:")
                for i, (q, a) in enumerate(self.history[-5:], 1):  # Show last 5
                    print(f"{i}. Q: {q[:50]}...")
                    print(f"   A: {a[:50]}...")
                continue
            elif query.lower() == 'clear':
                self.history.clear()
                print("Conversation history cleared.")
                continue
            
            if not query:
                continue
            
            # Create prompt with context
            context = ""
            if self.history:
                # Add last 2 exchanges for context
                recent_history = self.history[-2:]
                context = "Previous conversation:\n"
                for q, a in recent_history:
                    context += f"User: {q}\nAssistant: {a}\n"
                context += "\n"
            
            prompt = f"{context}### Instruction:\nProvide information about {query}.\n\n### Response:\n"
            
            try:
                # Generate response with streaming
                full_response = self.generate_streaming(prompt, max_length=300)
                
                # Filter and check safety
                answer = self.filter_response(full_response)
                is_safe, safety_msg = self.check_safety(answer)
                
                if not is_safe:
                    print(f"Warning: {safety_msg}")
                    answer = "I apologize, but I cannot provide information on that topic."
                
                # Add to history
                self.history.append((query, answer))
                if len(self.history) > self.max_history:
                    self.history.pop(0)
                    
            except Exception as e:
                print(f"Error generating response: {e}")
                answer = "I encountered an error. Please try again."
            
            # The response is already printed via streaming

if __name__ == "__main__":
    chat_interface = SafeChatInterface()
    chat_interface.chat()

## Training Restart

Safely deletes all training checkpoints and logs to allow restarting the training process from scratch.

In [ ]:
# restart_training.py
import os
import shutil

def restart_training():
    """Delete all training files to restart from scratch"""
    
    # Directories to delete
    dirs_to_delete = ["./wikipedia_model", "./logs"]
    
    for dir_path in dirs_to_delete:
        if os.path.exists(dir_path):
            print(f"Deleting {dir_path}...")
            shutil.rmtree(dir_path)
            print(f"Deleted {dir_path}")
        else:
            print(f"{dir_path} does not exist")
    
    print("\nAll training files deleted. You can now run the training cell to start fresh.")

if __name__ == "__main__":
    # Ask for confirmation
    confirm = input("This will delete all training checkpoints and logs. Are you sure? (yes/no): ")
    if confirm.lower() == 'yes':
        restart_training()
    else:
        print("Restart cancelled.")

## Hyperparameter Tuning

Automated hyperparameter optimization using grid search to find the best learning rate, batch size, and LoRA parameters for your model.

In [ ]:
# hyperparameter_tuning.py
import itertools
from transformers import TrainingArguments, Trainer
from datasets import load_from_disk
import torch
import os
import json
from tqdm import tqdm

# Load model and tokenizer
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

MODEL_OPTIONS = {
    "small": "TinyLlama/TinyLlama-1.1B-Chat-v1.0",  # 1.1B - easiest to train
    "medium": "microsoft/phi-2",  # 2.7B - good balance
    "large": "mistralai/Mistral-7B-v0.1"  # 7B - needs quantization
}

model_name = MODEL_OPTIONS["medium"]  # Start with Phi-2

# Quantization config for memory efficiency
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

print(f"Loading model: {model_name}")
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

print("Model and tokenizer loaded successfully!")

def objective(params):
    """Objective function for hyperparameter evaluation"""
    
    learning_rate, per_device_batch_size, lora_rank, lora_alpha, weight_decay = params
    
    # Load data
    dataset = load_from_disk("data/formatted_wikipedia")
    
    # Tokenize function
    def tokenize_function(examples):
        tokenized = tokenizer(examples["text"], truncation=True, padding="max_length", max_length=512)
        tokenized["labels"] = tokenized["input_ids"].copy()  # For causal LM, labels are the same as input_ids
        return tokenized
    
    # Tokenize datasets
    tokenized_train = dataset["train"].select(range(1000)).map(tokenize_function, batched=True)
    tokenized_test = dataset["test"].select(range(200)).map(tokenize_function, batched=True)
    
    # Set format for PyTorch
    tokenized_train.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
    tokenized_test.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
    
    # Calculate effective batch size
    gradient_accumulation_steps = max(1, 16 // per_device_batch_size)  # Target effective batch of 16
    
    # Training arguments
    training_args = TrainingArguments(
        output_dir="./grid_trials",
        num_train_epochs=1,  # Quick trial
        per_device_train_batch_size=per_device_batch_size,
        per_device_eval_batch_size=per_device_batch_size,
        gradient_accumulation_steps=gradient_accumulation_steps,
        learning_rate=learning_rate,
        weight_decay=weight_decay,
        fp16=True,
        save_steps=50,
        eval_steps=50,
        logging_steps=50,
        eval_strategy="steps",
        save_total_limit=1,
        load_best_model_at_end=True,
        metric_for_best_model="eval_loss",
        greater_is_better=False,
        seed=42,
        report_to=[],  # Disable integrations to avoid MLflow issues
        remove_unused_columns=False,  # Keep all columns
    )
    
    # Configure LoRA with parameters
    from peft import LoraConfig, get_peft_model
    lora_config = LoraConfig(
        r=lora_rank,
        lora_alpha=lora_alpha,
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
        lora_dropout=0.05,
        bias="none",
        task_type="CAUSAL_LM"
    )
    
    # Load model (assuming model and tokenizer are already loaded in notebook)
    model_with_lora = get_peft_model(model, lora_config)
    
    # Initialize trainer
    trainer = Trainer(
        model=model_with_lora,
        args=training_args,
        train_dataset=tokenized_train,
        eval_dataset=tokenized_test,
    )
    
    # Train
    trainer.train()
    
    # Get best eval loss
    best_eval_loss = trainer.state.best_metric
    
    # Cleanup
    del model_with_lora
    torch.cuda.empty_cache()
    
    return best_eval_loss, params

def run_hyperparameter_tuning():
    """Run grid search hyperparameter optimization"""
    print("Starting hyperparameter tuning with grid search...")
    print("This may take several hours depending on the number of combinations.")
    
    # Define parameter grid
    param_grid = {
        'learning_rate': [1e-4, 5e-4, 1e-3],
        'per_device_batch_size': [4, 8],
        'lora_rank': [8, 16],
        'lora_alpha': [16, 32],
        'weight_decay': [0.01, 0.05]
    }
    
    # Generate all combinations
    keys = param_grid.keys()
    values = param_grid.values()
    combinations = list(itertools.product(*values))
    
    print(f"Testing {len(combinations)} parameter combinations...")
    
    best_loss = float('inf')
    best_params = None
    
    for i, combo in enumerate(tqdm(combinations, desc="Hyperparameter Tuning", unit="trial")):
        params = dict(zip(keys, combo))
        print(f"\nTrial {i+1}/{len(combinations)}: {params}")
        
        try:
            loss, _ = objective(combo)
            print(f"Eval loss: {loss:.4f}")
            
            if loss < best_loss:
                best_loss = loss
                best_params = params
                print(f"New best loss: {best_loss:.4f} (Trial {i+1})")
                
        except Exception as e:
            print(f"Error in trial {i+1}: {e}")
            continue
    
    # Print results
    print("\n" + "="*50)
    print("HYPERPARAMETER TUNING RESULTS")
    print("="*50)
    print(f"Best eval loss: {best_loss:.4f}")
    print("\nBest hyperparameters:")
    for key, value in best_params.items():
        print(f"  {key}: {value}")
    
    # Save best parameters
    with open("best_hyperparams.json", "w") as f:
        json.dump(best_params, f, indent=2)
    
    print(f"\nBest parameters saved to best_hyperparams.json")
    print("Use these parameters in your main training script.")
    
    return best_params

# Uncomment to run tuning (takes significant time and resources)
if __name__ == "__main__":
    best_params = run_hyperparameter_tuning()

## Model Evaluation

Comprehensive model evaluation with multiple metrics including BLEU, ROUGE, perplexity, and custom benchmarks.

In [ ]:
# model_evaluation.py
import torch
from transformers import pipeline
from datasets import load_from_disk
import numpy as np
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
import json
from tqdm import tqdm
import matplotlib.pyplot as plt

class ModelEvaluator:
    """Comprehensive model evaluation suite"""
    
    def __init__(self, model, tokenizer, device="cuda"):
        self.model = model
        self.tokenizer = tokenizer
        self.device = device
        self.generator = pipeline(
            "text-generation",
            model=model,
            tokenizer=tokenizer,
            device=device,
            max_new_tokens=100,
            temperature=0.7,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )
        self.rouge_scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
        self.smooth = SmoothingFunction().method4
    
    def calculate_perplexity(self, texts, batch_size=8):
        """Calculate perplexity on a set of texts"""
        perplexities = []
        
        for i in range(0, len(texts), batch_size):
            batch_texts = texts[i:i+batch_size]
            
            # Tokenize
            encodings = self.tokenizer(batch_texts, return_tensors="pt", padding=True, truncation=True)
            input_ids = encodings.input_ids.to(self.device)
            attention_mask = encodings.attention_mask.to(self.device)
            
            with torch.no_grad():
                outputs = self.model(input_ids, attention_mask=attention_mask, labels=input_ids)
                loss = outputs.loss
                perplexity = torch.exp(loss).item()
                perplexities.append(perplexity)
        
        return np.mean(perplexities)
    
    def calculate_bleu_rouge(self, generated_texts, reference_texts):
        """Calculate BLEU and ROUGE scores"""
        bleu_scores = []
        rouge1_scores = []
        rouge2_scores = []
        rougeL_scores = []
        
        for gen, ref in zip(generated_texts, reference_texts):
            # BLEU score
            gen_tokens = gen.split()
            ref_tokens = ref.split()
            bleu = sentence_bleu([ref_tokens], gen_tokens, smoothing_function=self.smooth)
            bleu_scores.append(bleu)
            
            # ROUGE scores
            rouge_scores = self.rouge_scorer.score(ref, gen)
            rouge1_scores.append(rouge_scores['rouge1'].fmeasure)
            rouge2_scores.append(rouge_scores['rouge2'].fmeasure)
            rougeL_scores.append(rouge_scores['rougeL'].fmeasure)
        
        return {
            'bleu': np.mean(bleu_scores),
            'rouge1': np.mean(rouge1_scores),
            'rouge2': np.mean(rouge2_scores),
            'rougeL': np.mean(rougeL_scores)
        }
    
    def generate_samples(self, prompts, num_samples=10):
        """Generate text samples for qualitative evaluation"""
        samples = []
        
        for prompt in tqdm(prompts[:num_samples], desc="Generating samples"):
            try:
                outputs = self.generator(
                    prompt,
                    max_new_tokens=100,
                    num_return_sequences=1,
                    temperature=0.8,
                    do_sample=True,
                    pad_token_id=self.tokenizer.eos_token_id
                )
                generated_text = outputs[0]['generated_text']
                samples.append({
                    'prompt': prompt,
                    'generated': generated_text,
                    'length': len(generated_text.split())
                })
            except Exception as e:
                print(f"Error generating for prompt: {e}")
                samples.append({
                    'prompt': prompt,
                    'generated': "Error generating text",
                    'length': 0
                })
        
        return samples
    
    def benchmark_model(self, test_dataset=None, num_samples=100):
        """Run comprehensive benchmark"""
        print("Starting comprehensive model evaluation...")
        
        if test_dataset is None:
            # Load test dataset
            dataset = load_from_disk("data/formatted_wikipedia")
            test_dataset = dataset["test"]
        
        # Sample test data
        test_texts = test_dataset.select(range(min(num_samples, len(test_dataset))))
        test_prompts = [item['text'][:200] + "..." for item in test_texts]  # Use first 200 chars as prompts
        
        # Calculate perplexity
        print("Calculating perplexity...")
        perplexity = self.calculate_perplexity([item['text'] for item in test_texts])
        
        # Generate samples for BLEU/ROUGE
        print("Generating samples for BLEU/ROUGE evaluation...")
        generated_samples = self.generate_samples(test_prompts, num_samples=50)
        
        # Extract generated and reference texts
        generated_texts = [sample['generated'] for sample in generated_samples]
        reference_texts = [item['text'] for item in test_texts[:50]]
        
        # Calculate BLEU and ROUGE
        print("Calculating BLEU and ROUGE scores...")
        text_metrics = self.calculate_bleu_rouge(generated_texts, reference_texts)
        
        # Compile results
        results = {
            'perplexity': perplexity,
            'bleu_score': text_metrics['bleu'],
            'rouge1_score': text_metrics['rouge1'],
            'rouge2_score': text_metrics['rouge2'],
            'rougeL_score': text_metrics['rougeL'],
            'samples': generated_samples[:5],  # Save first 5 samples
            'num_samples_evaluated': len(generated_samples)
        }
        
        # Print results
        print("\n" + "="*60)
        print("MODEL EVALUATION RESULTS")
        print("="*60)
        print(f"Perplexity: {results['perplexity']:.4f}")
        print(f"BLEU Score: {results['bleu_score']:.4f}")
        print(f"ROUGE-1 Score: {results['rouge1_score']:.4f}")
        print(f"ROUGE-2 Score: {results['rouge2_score']:.4f}")
        print(f"ROUGE-L Score: {results['rougeL_score']:.4f}")
        print(f"Samples evaluated: {results['num_samples_evaluated']}")
        
        # Save results
        with open("evaluation_results.json", "w") as f:
            # Convert numpy types to native Python types for JSON serialization
            json_results = {
                k: float(v) if isinstance(v, np.floating) else v 
                for k, v in results.items() 
                if k != 'samples'
            }
            json_results['samples'] = results['samples']
            json.dump(json_results, f, indent=2)
        
        print("Results saved to evaluation_results.json")
        
        return results
    
    def plot_evaluation_history(self, checkpoint_dirs=None):
        """Plot evaluation metrics across checkpoints"""
        if checkpoint_dirs is None:
            checkpoint_dirs = [d for d in os.listdir("./wikipedia_model") if d.startswith("checkpoint-")]
            checkpoint_dirs.sort(key=lambda x: int(x.split("-")[1]))
        
        perplexities = []
        steps = []
        
        for checkpoint in checkpoint_dirs:
            checkpoint_path = f"./wikipedia_model/{checkpoint}"
            if os.path.exists(f"{checkpoint_path}/evaluation_results.json"):
                with open(f"{checkpoint_path}/evaluation_results.json", "r") as f:
                    results = json.load(f)
                    perplexities.append(results.get('perplexity', 0))
                    steps.append(int(checkpoint.split("-")[1]))
        
        if perplexities:
            plt.figure(figsize=(10, 6))
            plt.plot(steps, perplexities, marker='o')
            plt.xlabel('Training Steps')
            plt.ylabel('Perplexity')
            plt.title('Model Perplexity Across Training Checkpoints')
            plt.grid(True)
            plt.savefig('evaluation_history.png')
            plt.show()
        else:
            print("No evaluation results found in checkpoints")

# Usage example
def run_model_evaluation():
    """Run comprehensive model evaluation"""
    evaluator = ModelEvaluator(model, tokenizer)
    results = evaluator.benchmark_model()
    
    # Plot evaluation history if multiple checkpoints exist
    evaluator.plot_evaluation_history()
    
    return results

# Uncomment to run evaluation
# if __name__ == "__main__":
#     results = run_model_evaluation()

## Model Deployment

Export and deploy your trained model for production use with ONNX, quantization, and containerization.

In [ ]:
# model_deployment.py
import torch
from transformers import AutoTokenizer
import onnxruntime as ort
from onnxruntime.quantization import quantize_dynamic, QuantType
import os
import json
from pathlib import Path

class ModelDeployer:
    """Model deployment utilities for production"""
    
    def __init__(self, model, tokenizer, output_dir="./deployed_model"):
        self.model = model
        self.tokenizer = tokenizer
        self.output_dir = Path(output_dir)
        self.output_dir.mkdir(exist_ok=True)
    
    def merge_and_save_model(self):
        """Merge LoRA weights with base model and save"""
        print("Merging LoRA weights with base model...")
        
        # Merge LoRA weights
        merged_model = self.model.merge_and_unload()
        
        # Save merged model
        merged_dir = self.output_dir / "merged_model"
        merged_dir.mkdir(exist_ok=True)
        
        merged_model.save_pretrained(merged_dir)
        self.tokenizer.save_pretrained(merged_dir)
        
        print(f"Merged model saved to {merged_dir}")
        return merged_dir
    
    def export_to_onnx(self, merged_model_path=None, opset_version=14):
        """Export model to ONNX format"""
        print("Exporting model to ONNX format...")
        
        if merged_model_path is None:
            merged_model_path = self.merge_and_save_model()
        
        from transformers.onnx import export
        from transformers import AutoModelForCausalLM
        
        # Load merged model
        model = AutoModelForCausalLM.from_pretrained(
            merged_model_path,
            torch_dtype=torch.float16,
            device_map="auto"
        )
        
        # Export to ONNX
        onnx_path = self.output_dir / "model.onnx"
        
        # Create dummy input
        dummy_input = self.tokenizer("Hello world", return_tensors="pt")
        
        # Export
        torch.onnx.export(
            model,
            (dummy_input["input_ids"], dummy_input["attention_mask"]),
            onnx_path,
            opset_version=opset_version,
            input_names=["input_ids", "attention_mask"],
            output_names=["logits"],
            dynamic_axes={
                "input_ids": {0: "batch_size", 1: "sequence_length"},
                "attention_mask": {0: "batch_size", 1: "sequence_length"},
                "logits": {0: "batch_size", 1: "sequence_length"}
            }
        )
        
        print(f"ONNX model exported to {onnx_path}")
        return onnx_path
    
    def quantize_onnx_model(self, onnx_path=None):
        """Quantize ONNX model for better performance"""
        print("Quantizing ONNX model...")
        
        if onnx_path is None:
            onnx_path = self.output_dir / "model.onnx"
        
        quantized_path = self.output_dir / "model_quantized.onnx"
        
        quantize_dynamic(
            model_input=str(onnx_path),
            model_output=str(quantized_path),
            weight_type=QuantType.QInt8
        )
        
        print(f"Quantized model saved to {quantized_path}")
        return quantized_path
    
    def create_dockerfile(self):
        """Create Dockerfile for containerized deployment"""
        dockerfile_content = '''FROM python:3.9-slim

# Install system dependencies
RUN apt-get update && apt-get install -y \\
    build-essential \\
    && rm -rf /var/lib/apt/lists/*

# Set working directory
WORKDIR /app

# Copy requirements
COPY requirements.txt .
RUN pip install --no-cache-dir -r requirements.txt

# Copy model files
COPY . .

# Expose port
EXPOSE 8000

# Run the application
CMD ["python", "app.py"]
'''
        
        dockerfile_path = self.output_dir / "Dockerfile"
        with open(dockerfile_path, "w") as f:
            f.write(dockerfile_content)
        
        print(f"Dockerfile created at {dockerfile_path}")
        return dockerfile_path
    
    def create_requirements_txt(self):
        """Create requirements.txt for deployment"""
        requirements = '''transformers>=4.21.0
torch>=1.12.0
accelerate>=0.20.0
fastapi>=0.95.0
uvicorn>=0.21.0
pydantic>=1.10.0
onnxruntime>=1.14.0
numpy>=1.21.0
'''
        
        req_path = self.output_dir / "requirements.txt"
        with open(req_path, "w") as f:
            f.write(requirements)
        
        print(f"requirements.txt created at {req_path}")
        return req_path
    
    def create_fastapi_app(self):
        """Create FastAPI application for serving the model"""
        app_content = '''from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from transformers import pipeline
import torch
import os

app = FastAPI(title="Wikipedia LLM API", version="1.0.0")

# Load model and tokenizer
MODEL_PATH = os.getenv("MODEL_PATH", "./merged_model")

try:
    generator = pipeline(
        "text-generation",
        model=MODEL_PATH,
        device=0 if torch.cuda.is_available() else -1,
        torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
        max_new_tokens=100,
        temperature=0.7,
        do_sample=True,
    )
    print("Model loaded successfully")
except Exception as e:
    print(f"Error loading model: {e}")
    generator = None

class GenerationRequest(BaseModel):
    prompt: str
    max_length: int = 100
    temperature: float = 0.7
    num_return_sequences: int = 1

class GenerationResponse(BaseModel):
    generated_texts: list[str]

@app.post("/generate", response_model=GenerationResponse)
async def generate_text(request: GenerationRequest):
    if generator is None:
        raise HTTPException(status_code=500, detail="Model not loaded")
    
    try:
        outputs = generator(
            request.prompt,
            max_new_tokens=request.max_length,
            temperature=request.temperature,
            num_return_sequences=request.num_return_sequences,
            pad_token_id=generator.tokenizer.eos_token_id
        )
        
        generated_texts = [output["generated_text"] for output in outputs]
        return GenerationResponse(generated_texts=generated_texts)
    
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Generation failed: {str(e)}")

@app.get("/health")
async def health_check():
    return {"status": "healthy", "model_loaded": generator is not None}

if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host="0.0.0.0", port=8000)
'''
        
        app_path = self.output_dir / "app.py"
        with open(app_path, "w") as f:
            f.write(app_content)
        
        print(f"FastAPI app created at {app_path}")
        return app_path
    
    def create_deployment_package(self):
        """Create complete deployment package"""
        print("Creating deployment package...")
        
        # Merge and save model
        merged_path = self.merge_and_save_model()
        
        # Create requirements.txt
        self.create_requirements_txt()
        
        # Create FastAPI app
        self.create_fastapi_app()
        
        # Create Dockerfile
        self.create_dockerfile()
        
        # Create README
        readme_content = '''# Wikipedia LLM Deployment

This directory contains the deployment package for the fine-tuned Wikipedia LLM.

## Local Deployment

1. Install dependencies:
   ```bash
   pip install -r requirements.txt
   ```

2. Run the API server:
   ```bash
   python app.py
   ```

3. Test the API:
   ```bash
   curl -X POST "http://localhost:8000/generate" \\
        -H "Content-Type: application/json" \\
        -d '{"prompt": "The history of artificial intelligence", "max_length": 50}'
   ```

## Docker Deployment

1. Build the Docker image:
   ```bash
   docker build -t wikipedia-llm .
   ```

2. Run the container:
   ```bash
   docker run -p 8000:8000 wikipedia-llm
   ```

## API Endpoints

- `POST /generate`: Generate text from a prompt
- `GET /health`: Health check endpoint

## Model Information

- Base model: Microsoft Phi-2
- Fine-tuned on: Wikipedia dataset
- Framework: Transformers
- Quantization: 4-bit (during training)
'''
        
        readme_path = self.output_dir / "README.md"
        with open(readme_path, "w") as f:
            f.write(readme_content)
        
        print(f"Deployment package created in {self.output_dir}")
        print("Files created:")
        print("  - merged_model/ (merged model weights)")
        print("  - requirements.txt")
        print("  - app.py (FastAPI application)")
        print("  - Dockerfile")
        print("  - README.md")
        
        return self.output_dir

def deploy_model():
    """Complete model deployment workflow"""
    deployer = ModelDeployer(model, tokenizer)
    
    # Create deployment package
    deployment_dir = deployer.create_deployment_package()
    
    print(f"\nDeployment package ready at: {deployment_dir}")
    print("\nNext steps:")
    print("1. Test locally: python app.py")
    print("2. Build Docker: docker build -t wikipedia-llm .")
    print("3. Deploy to cloud platform of choice")
    
    return deployment_dir

# Uncomment to create deployment package
# if __name__ == "__main__":
#     deploy_model()

## Data Analysis

Analyze your training dataset statistics, distribution, and quality metrics to understand your model's training data.

In [ ]:
# data_analysis.py
import matplotlib.pyplot as plt
import seaborn as sns
from datasets import load_from_disk
import numpy as np
from collections import Counter
import re
from tqdm import tqdm
import pandas as pd
from wordcloud import WordCloud
import json
import os

class DataAnalyzer:
    """Comprehensive dataset analysis tools"""
    
    def __init__(self, dataset_path="data/formatted_wikipedia"):
        try:
            self.dataset = load_from_disk(dataset_path)
            print(f"Loaded dataset with {len(self.dataset)} splits")
            
            # Debug: Check dataset structure
            if len(self.dataset) > 0:
                first_split = list(self.dataset.keys())[0]
                first_item = self.dataset[first_split][0]
                print(f"First item type: {type(first_item)}")
                if isinstance(first_item, dict):
                    print(f"First item keys: {list(first_item.keys())}")
                else:
                    print(f"First item (first 100 chars): {str(first_item)[:100]}")
        except Exception as e:
            print(f"Error loading dataset: {e}")
            self.dataset = None
    
    def basic_statistics(self):
        """Calculate basic dataset statistics"""
        if self.dataset is None:
            return None
        
        stats = {}
        
        for split_name, split_data in self.dataset.items():
            # Handle both dict format {'text': '...'} and string format
            try:
                # Try dict format first
                texts = [item['text'] for item in split_data]
            except (TypeError, KeyError):
                # If that fails, assume items are strings directly
                texts = [item for item in split_data if isinstance(item, str)]
            
            # Text lengths
            text_lengths = [len(text.split()) for text in texts]
            
            stats[split_name] = {
                'num_samples': len(texts),
                'avg_words': np.mean(text_lengths),
                'min_words': np.min(text_lengths),
                'max_words': np.max(text_lengths),
                'median_words': np.median(text_lengths),
                'total_words': sum(text_lengths),
                'avg_chars': np.mean([len(text) for text in texts])
            }
        
        # Print statistics
        print("\n" + "="*60)
        print("DATASET STATISTICS")
        print("="*60)
        
        for split, stat in stats.items():
            print(f"\n{split.upper()} SPLIT:")
            print(f"  Samples: {stat['num_samples']:,}")
            print(f"  Avg words per sample: {stat['avg_words']:.1f}")
            print(f"  Word count range: {stat['min_words']}-{stat['max_words']}")
            print(f"  Median words: {stat['median_words']:.1f}")
            print(f"  Total words: {stat['total_words']:,}")
            print(f"  Avg characters per sample: {stat['avg_chars']:.1f}")
        
        return stats
    
    def plot_distributions(self, stats):
        """Plot data distributions"""
        if stats is None:
            return
        
        fig, axes = plt.subplots(2, 2, figsize=(15, 10))
        fig.suptitle('Dataset Analysis', fontsize=16)
        
        splits = list(stats.keys())
        colors = ['blue', 'green', 'red', 'orange']
        
        # Sample counts
        axes[0, 0].bar(splits, [stats[s]['num_samples'] for s in splits], color=colors[:len(splits)])
        axes[0, 0].set_title('Number of Samples per Split')
        axes[0, 0].set_ylabel('Count')
        
        # Average words
        axes[0, 1].bar(splits, [stats[s]['avg_words'] for s in splits], color=colors[:len(splits)])
        axes[0, 1].set_title('Average Words per Sample')
        axes[0, 1].set_ylabel('Words')
        
        # Word length distributions
        for i, split in enumerate(splits):
            # Handle both dict format {'text': '...'} and string format
            try:
                # Try dict format first
                texts = [item['text'] for item in self.dataset[split]]
            except (TypeError, KeyError):
                # If that fails, assume items are strings directly
                texts = [item for item in self.dataset[split] if isinstance(item, str)]
            lengths = [len(text.split()) for text in texts[:1000]]  # Sample for plotting
            axes[1, 0].hist(lengths, alpha=0.7, label=split, bins=30, color=colors[i])
        
        axes[1, 0].set_title('Word Length Distribution (Sample)')
        axes[1, 0].set_xlabel('Words per Sample')
        axes[1, 0].set_ylabel('Frequency')
        axes[1, 0].legend()
        
        # Character length distributions
        for i, split in enumerate(splits):
            # Handle both dict format {'text': '...'} and string format
            try:
                # Try dict format first
                texts = [item['text'] for item in self.dataset[split]]
            except (TypeError, KeyError):
                # If that fails, assume items are strings directly
                texts = [item for item in self.dataset[split] if isinstance(item, str)]
            lengths = [len(text) for text in texts[:1000]]  # Sample for plotting
            axes[1, 1].hist(lengths, alpha=0.7, label=split, bins=30, color=colors[i])
        
        axes[1, 1].set_title('Character Length Distribution (Sample)')
        axes[1, 1].set_xlabel('Characters per Sample')
        axes[1, 1].set_ylabel('Frequency')
        axes[1, 1].legend()
        
        plt.tight_layout()
        plt.savefig('data_analysis/dataset_analysis.png', dpi=300, bbox_inches='tight')
        plt.show()
    
    def analyze_vocabulary(self, num_top_words=20):
        """Analyze vocabulary and word frequencies"""
        if self.dataset is None:
            return None
        
        print("\nAnalyzing vocabulary...")
        
        # Combine all texts
        all_texts = []
        for split_data in self.dataset.values():
            # Handle both dict format {'text': '...'} and string format
            try:
                # Try dict format first
                texts = [item['text'] for item in split_data]
            except (TypeError, KeyError):
                # If that fails, assume items are strings directly
                texts = [item for item in split_data if isinstance(item, str)]
            all_texts.extend(texts)
        
        # Tokenize and count words
        word_counts = Counter()
        
        for text in tqdm(all_texts[:5000], desc="Processing texts"):  # Sample for speed
            # Simple tokenization (split on whitespace and remove punctuation)
            words = re.findall(r'\b\w+\b', text.lower())
            word_counts.update(words)
        
        # Get top words
        top_words = word_counts.most_common(num_top_words)
        
        print(f"\nTop {num_top_words} most frequent words:")
        for word, count in top_words:
            print(f"  {word}: {count:,}")
        
        # Vocabulary size
        vocab_size = len(word_counts)
        print(f"\nTotal unique words (vocabulary size): {vocab_size:,}")
        
        return {
            'top_words': top_words,
            'vocab_size': vocab_size,
            'word_counts': word_counts
        }
    
    def create_wordcloud(self, word_counts, max_words=100):
        """Create word cloud visualization"""
        # Filter out common stop words
        stop_words = {'the', 'a', 'an', 'and', 'or', 'but', 'in', 'on', 'at', 'to', 'for', 'of', 'with', 'by', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'do', 'does', 'did', 'will', 'would', 'could', 'should', 'may', 'might', 'must', 'can', 'this', 'that', 'these', 'those', 'i', 'you', 'he', 'she', 'it', 'we', 'they', 'me', 'him', 'her', 'us', 'them'}
        
        filtered_words = {word: count for word, count in word_counts.items() 
                         if word not in stop_words and len(word) > 2}
        
        # Create word cloud
        wordcloud = WordCloud(
            width=800, 
            height=400, 
            background_color='white',
            max_words=max_words,
            colormap='viridis'
        ).generate_from_frequencies(filtered_words)
        
        # Plot
        plt.figure(figsize=(15, 8))
        plt.imshow(wordcloud, interpolation='bilinear')
        plt.axis('off')
        plt.title('Word Cloud of Training Data', fontsize=16)
        plt.savefig('data_analysis/wordcloud.png', dpi=300, bbox_inches='tight')
        plt.show()
    
    def export_analysis_report(self, stats, vocab_info):
        """Export comprehensive analysis report"""
        report = {
            'dataset_statistics': stats,
            'vocabulary_analysis': {
                'vocab_size': vocab_info['vocab_size'],
                'top_words': vocab_info['top_words'][:10]
            },
            'recommendations': []
        }
        
        # Generate recommendations
        if stats:
            total_samples = sum(s['num_samples'] for s in stats.values())
            if total_samples < 10000:
                report['recommendations'].append("Consider increasing dataset size for better model performance")
            
            avg_words = np.mean([s['avg_words'] for s in stats.values()])
            if avg_words < 50:
                report['recommendations'].append("Texts are quite short - consider longer passages for better context")
        
        if vocab_info and vocab_info['vocab_size'] < 10000:
            report['recommendations'].append("Limited vocabulary - model may struggle with diverse topics")
        
        # Save report
        with open('data_analysis/data_analysis_report.json', 'w') as f:
            json.dump(report, f, indent=2, default=str)
        
        print("\nAnalysis report saved to data_analysis/data_analysis_report.json")
        
        return report

def run_complete_analysis():
    """Run complete dataset analysis"""
    os.makedirs("data_analysis", exist_ok=True)
    analyzer = DataAnalyzer()
    
    # Basic statistics
    stats = analyzer.basic_statistics()
    
    # Plot distributions
    analyzer.plot_distributions(stats)
    
    # Vocabulary analysis
    vocab_info = analyzer.analyze_vocabulary()
    
    # Create word cloud
    if vocab_info:
        analyzer.create_wordcloud(vocab_info['word_counts'])
    
    # Export report
    report = analyzer.export_analysis_report(stats, vocab_info)
    
    print("\nAnalysis complete! Check the generated plots and report in the data_analysis folder.")
    
    return report

if __name__ == "__main__":
    run_complete_analysis()